# P03_05_Multivar_LSTM

In [1]:
section_fig_main_title = 'P03_05_Multivar_LSTM_'

In [2]:
!which python

/Users/amin/miniconda3/bin/python


In [3]:
import os, sys
import requests
from urllib.parse import urljoin, urlparse
# from bs4 import BeautifulSoup as bs
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
from pandas.plotting import lag_plot

from tensorflow import keras

ModuleNotFoundError: No module named 'scipy.optimize._highs._highs_wrapper'

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
# !/Users/amin/Desktop/proj/env/bin/python -m pip install xgboost

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error

In [ ]:
# !/Users/amin/anaconda3/bin/python -m pip install --upgrade tensorflow 
# pip install --upgrade tensorflow 

In [ ]:

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD

import math
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
from pandas.plotting import autocorrelation_plot
import gif

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error

In [ ]:
import tensorflow as tf
tf.__version__


In [ ]:
import gif
from tqdm import tqdm

In [ ]:
from sklearn import metrics
import statsmodels.api as sm

## Read Data 

Using the univariate evenly spaced time series data from P03_01.

In [ ]:
csv_path = "../Data/Data_ICME_Edited_Decoded_out_P02_03.csv"
df = pd.read_csv(csv_path)

In [ ]:
df.index = df['ICME Plasma/Field Start Y/M/D (UT) (b)']

In [ ]:
df.head()

In [ ]:
df.plot(figsize=(20,6))

In [ ]:
df.columns

In [ ]:
keep_cols = ['ICME Plasma/Field Start Y/M/D (UT) (b)', 'BDE? (e)', 'Qual. (g)',
       'dV (km/s) (h)', 'V_ICME (km/s) (i)', 'B (nT) (k)',
       'MC? (l)', 'Dst (nT) (m)', 'Normalized_time_interval']
df = df[keep_cols]
df.head()

In [ ]:
df.info()

## Split the data

In [ ]:
split_fraction = 0.69
train_split = int(split_fraction * int(df.shape[0]))
step = 1

past = 60
future = 1
learning_rate = 0.001
batch_size = 256
epochs = 10

def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [ ]:
selected_features = ['BDE? (e)', 'Qual. (g)',
                   'dV (km/s) (h)', 'V_ICME (km/s) (i)', 'B (nT) (k)',
                   'MC? (l)', 'Dst (nT) (m)', 'Normalized_time_interval']

features = df[selected_features]
features.index = df.index

In [ ]:
features.head()

In [ ]:
features.shape

In [ ]:
df.columns

In [ ]:
titles = ['BDE? (e)', 'Qual. (g)', 'dV (km/s) (h)', 'V_ICME (km/s) (i)',
       'B (nT) (k)', 'MC? (l)', 'Dst (nT) (m)', 'Normalized_time_interval']

feature_keys = ['BDE? (e)', 'Qual. (g)', 'dV (km/s) (h)', 'V_ICME (km/s) (i)',
       'B (nT) (k)', 'MC? (l)', 'Dst (nT) (m)', 'Normalized_time_interval']

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
]

date_time_key = "ICME Plasma/Field Start Y/M/D (UT) (b)"


def show_raw_visualization(data):
    time_data = data[date_time_key]
    fig, axes = plt.subplots(
        nrows=4, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)):
        key = feature_keys[i]
        c = colors[i % (len(colors))]
        t_data = data[key]
        t_data.index = time_data
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{} - {}".format(titles[i], key),
            rot=25,
        )
        ax.legend([titles[i]])
    plt.tight_layout()


show_raw_visualization(df)

## Normalizing data

In [ ]:
features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

In [ ]:
features.shape

## Train and Val

In [ ]:
train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]

In [ ]:
train_data.shape

In [ ]:
val_data.shape

In [ ]:
train_data.head()

In [ ]:
start = past + future
end = start + train_split
print('start = ', start)
print('end = ', end)
print('train_split = ', train_split)
sequence_length = int(past / step)
print('sequence_length = ', sequence_length)

## Choose target column

In [ ]:
features.iloc[start:end][[3]]

In [ ]:
x_train = train_data[[i for i in range(7)]].values
y_train = features.iloc[start:end][[3]]

In [ ]:
y_train.plot()

In [ ]:
len(x_train)

In [ ]:
print('x_train.shape', x_train.shape,
      '\ny_train.shape', y_train.shape)

## Preprocessing data

In [ ]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(x_train,
                                                                  y_train,
                                                                  sequence_length=sequence_length,
                                                                  sampling_rate=step,
                                                                  batch_size=batch_size,
                                                                  )

In [ ]:
# dataset_train
# print(val_data)

In [ ]:
x_end = len(val_data) - past - future
# print('x_end ', x_end)
label_start = train_split + past + future
# print('label_start ', label_start)

In [ ]:
x_val = val_data.iloc[:x_end][[i for i in range(7)]].values
y_val = features.iloc[label_start:][[3]]

In [ ]:
y_val.plot()

In [ ]:
# train_split
# x_val.shape
# y_val.shape
# sequence_length
# batch_size
# step

In [ ]:
dataset_val = keras.preprocessing.timeseries_dataset_from_array(x_val,
                                                                y_val,
                                                                sequence_length=sequence_length,
                                                                sampling_rate=step,
                                                                batch_size=batch_size,
                                                                )


for batch in dataset_train.take(1):
    inputs, targets = batch

# print("Input shape:", inputs.numpy().shape)
# print("Target shape:", targets.numpy().shape)


print("Input shape:", inputs.shape)
print("Target shape:", targets.shape)

## Assemble keras model

In [ ]:
inputs = keras.layers.Input(shape=(inputs.shape[1], 
                                   inputs.shape[2]))

lstm_out = keras.layers.LSTM(8)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              loss="mse")

model.summary()

## Fit the model

In [ ]:
path_checkpoint = "../Models/LSTM_Multivariate_P03_05.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", 
                                            min_delta=0, 
                                            patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
                                                    monitor="val_loss",
                                                    filepath=path_checkpoint,
                                                    verbose=1,
                                                    #save_weights_only=True,
                                                    save_best_only=True,
                                                )

history = model.fit(
                    dataset_train,
                    epochs=20,
                    validation_data=dataset_val,
                    shuffle=False,
                    callbacks=[modelckpt_callback],#es_callback, 
                )

## Load the pre-trained model

In [ ]:
from tensorflow.keras.models import *

In [ ]:
path_1 = '../Models/LSTM_Multivariate_P03_05.h5'
model = keras.models.load_model(path_1)

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


visualize_loss(history, "Training and Validation Loss")

In [ ]:
def show_plot(plot_data, delta, title):
    plt.figure(figsize=(20,8))
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
            
            
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
#     plt.show()
    
    return

## GIF

In [ ]:
@gif.frame
def plott(i):
    real_values = []
    predited_values = []

    for x, y in dataset_val.take(-1):



#         for i in range(1,33):
#             print(i)
        show_plot(
            [x[i][:, 3].numpy(), y[i].numpy(), model.predict(x)[i]],
            1,
            "Single Step Prediction",
        )

        real_values.append(y[i].numpy())
        predited_values.append(model.predict(x)[i])
        
        
        
        

In [ ]:
plott(1)

In [ ]:
frames = []    
for i in tqdm(range(1,33)):

    
    frame = plott(i)
    frames.append(frame)

In [ ]:
gif.save(frames, "../Figures/LSTM_multi_P3_5.gif", duration=2)

<p align="center">
  <img width="1200" src="../Figures/LSTM_multi_P3_5.gif" >
</p>

In [ ]:
from sklearn.metrics import r2_score

coefficient_of_dermination = r2_score(real_values, predited_values)
coefficient_of_dermination

In [ ]:
rs2 = round(coefficient_of_dermination,2)
rs2

In [ ]:
plt.figure(figsize=(10,10))

plt.scatter(real_values, predited_values, color='r', 
            alpha=0.4, label=f'Validation Data with R2 = {rs2}',s=100)
plt.xlabel("Normalized Real Temperature", fontsize=14)
plt.ylabel("Normalized Predited Temperature", fontsize=14)
plt.xticks(fontsize=12), plt.yticks(fontsize=12)
plt.axline([0, 0], [1, 1], color ='k')
plt.hlines(0, -4, 4, linestyles="--", color ='k')
plt.vlines(0, -4, 4, linestyles="--", color ='k')

plt.xlim(-3,3), plt.ylim(-3,3)

plt.title("LSTM model Temperature forcast results for 12 \
hours ahead\nData from 120 hours lags is used for prediction", fontsize=14)

plt.tight_layout()
plt.legend(fontsize=14)

# plt.savefig("../Figures/" + section_fig_main_title + "LSTM12HoursAhead.png")